# Dimensionality Reduction I, PCA

We start off dimensionality reduction by discussing perhaps the most popular dimensionaliy reduction technique in data science, principal components analysis (PCA).

## What You'll Accomplish

We'll:
<ul>
    <li>Briefly discuss the need or desire for dimensionality reduction,</li>
    <li>Introduce the concept and mathematics behind PCA,</li>
    <li>Implement PCA on the Wisconsin cancer set,</li>
    <li>Show how to interpret the output of PCA by examining the component vectors with basketball data,</li>
    <li>Introduce the explained variance curve.</li>
</ul>

Let's go!

In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

## But Why Dimensionality Reduction?

There are a number of reasons! Here are a few:

1. Reducing the number of dimensions is a way we can eliminate noise variables from our data allowing for better, we'll see an example of this with the Wisconsin Cancer data.

2. Your data is too big. This is common with image classification problems like the MNIST data.

3. Humans can only see in 2 or 3 dimensions, and interesting data sets often have many more dimensions than that.


I've hopefully convinced you of the importance of dimensionality reduction. If not, well that's too bad, humor me anyway.

## Principal Components Analysis (PCA)

PCA is one of the most popular dimensionality reduction techniques. So much so that we may take an entire day on just this topic, sorry manifold learning and t-SNE.

This technique has been around since Karl Pearson's 1903 paper, <a href = "https://www.tandfonline.com/doi/abs/10.1080/14786440109462720"><i>On lines and planes of closest fit to systems of points in space</i></a>.

### The Intuition

When you reduce the dimension of a data set you are inherently losing information. Therefore you want to ensure that you do it in a way that "retains as much information as possible". PCA tackles this problem in a very statistical manner.

There's an idea in statistics that the information of a data set is located within that data set's variation. Thus when you reduce the dimension of a data set, you want to project your data onto a lower dimensional space that captures as much of the original variance in the data as possible. Thinking in terms of optimization, your goal is to project into a lower dimensional hyperplane in a way that maximizes variance.

Here's a heuristic algorithm:
1. Center your data so that each feature has 0 mean, this is done for convenience.
2. Find the direction in space along which projections have the highest variance, this produces the first principal component.
3. Find the direction orthogonal to the first principal component that maximizes variance, this is the second principal component.
4. Continue in this way, the kth principal component is the variance-maximizing direction orthogonal to the previous k-1 components.

Let's see what we mean in a 2-D example:

In [ ]:
# Make some random data
np.random.seed(440)

x1 = 8*np.random.randn(500)
x2 = 3*np.random.randn(500)

X = np.concatenate([x1.reshape(-1,1),x2.reshape(-1,1)], axis = 1)

angle = -np.pi/4

X = X.dot(np.array([[np.cos(angle),-np.sin(angle)],[np.sin(angle),np.cos(angle)]]))


plt.figure(figsize=(10,8))
plt.scatter(X[:, 0], X[:, 1])

plt.xlabel("$x_1$", fontsize=14)
plt.ylabel("$x_2$", fontsize=14)

plt.axis('equal')

plt.show()

In [ ]:
# PCA is stored in decomposition
from sklearn.decomposition import PCA

In [ ]:
# make the PCA object
# we'll project down to 2-D
pca = PCA(n_components = 2)

# Fit the data
pca.fit(X)

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, 
                    shrinkB=0,
                    color="black")
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.figure(figsize=(10,8))
plt.scatter(X[:, 0], X[:, 1], alpha=.6)

for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
    
    
plt.xlabel("$x_1$", fontsize=14)
plt.ylabel("$x_2$", fontsize=14)

plt.axis('equal')
plt.show()

### The Math, The Myth, The Legend

Now let's get down to the mathematical details. We consider the maximal variance formulation of the problem, for those interested in the original problem statement proposed by Pearson check out the Homework.

Suppose we have $n$ observations of $m$ features, let $X_1,X_2,\dots,X_m$ be $n$ by $1$ vectors containing the observations of each of the $m$ features. And for ease of notation assume each has been centered to have mean $0$.

We'll restrict ourselves to the case of finding the first principal component, the others can be found in a similar fashion.

Let 
$$
X = \left(X_1 | X_2 | \dots |X_m \right)
$$
be an $n$ by $m$ feature matrix.

Our goal is to find $w=(w_1,w_2,\dots,w_m)^T$ so that $\text{Var}(w_1 X_1 + w_2 X_2 + \dots + w_m X_m) = \text{Var}(w^T X)$ is maximized. Because we have centered the columns of $X$ we have:
$$
\text{Var}(w^T X) = E(w^T X X^T w) = w^T E(X X^T) w = w^T \Sigma w,
$$
where $\Sigma$ is the covariance matrix of $X$. In order to ensure our $w$ is finite we impose the constraint that $||w|| = 1$. To solve the constrained optimization problem we use Lagrange multipliers where $f(w) = w^t \Sigma w$ and $g(w) = w^T w - 1$. Using matrix calculus (see the references below for a helpful resource):
$$
\partial_w \left(w^T \Sigma w - \lambda (w^T w - 1)\right) = 2 \Sigma w - 2\lambda w.
$$

Setting this equal to $0$ and solving gives
$$
\Sigma w = \lambda w,
$$
the standard eigenvalue setup.

So the vector $w$ that maximizes variance is an eigenvector corresponding to the largest eigenvector of the covariance matrix of $X$.

This vector is known as the first principal component, and as luck would have it the remaining principal components correspond to the remaining eigenvectors.



## Questions?


<br>
<br>
<br>

## End Questions

## Classifying Cancer Again

Now let's get some practice implementing PCA to help with supervised learning.

We return once again to classifying cancer.

In [ ]:
# import the data
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
X,y = load_breast_cancer(return_X_y = True)

# test train split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = .25,random_state = 614,shuffle = True,stratify = y)

In [ ]:
# Let's view the training data
# Note this may take a bit.
fig, axes = plt.subplots(15, 2, figsize = (12,30))

ax = axes.ravel()

for i in range(30):
    _, bins = np.histogram(X_train[:, i], bins = 50)
    ax[i].hist(X_train[y_train == 1, i], bins = bins, color = 'red', alpha = .5)
    ax[i].hist(X_train[y_train == 0, i], bins = bins, color = 'blue', alpha = .5)
    ax[i].set_title("feature " + str(i) + ": " + cancer.feature_names[i])
    ax[i].set_yticks(())

ax[0].legend(['malignant', 'benign'], loc = 'best')
    
fig.tight_layout()

Instead of using our intuition and explatory data analysis let's build a model implementing PCA as an added preprocessing step.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
pca_pipe = Pipeline([('scaler',StandardScaler()),
                ('pca',PCA(n_components=2))])

In [ ]:
X_pca = pca_pipe.fit_transform(X_train)

In [ ]:
plt.figure(figsize = (8,8))

plt.scatter(X_pca[y_train==0,0],X_pca[y_train==0,1],c='blue',label='benign')
plt.scatter(X_pca[y_train==1,0],X_pca[y_train==1,1],c='red',label='malignant')

plt.legend(fontsize=14)
plt.xlabel("pca 1",fontsize=16)
plt.ylabel("pca 2",fontsize=16)

plt.show()

## Practice

Combine PCA with one of our previous classification techniques to build a cancer classifier. If you have time implement cross validation and get a sense of how well we should expect this approach to work.

In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






## That's Nice, But What Does it Mean?

So we've seen that PCA can be quite useful. However, many of you might be wondering how we can interpret PCA results.

Well let's see.

### 3 > 2

We return to the basketball court. Basketball courts can be sliced into 15 distinct regions, you can then take all of a team's shot attempts and bin them according to the region where they were taken, if you then divide by their total shots taken you've got yourself a team's shot distribution. Below is an image of the court cut into 15 regions.

<img src="CourtZones.png" style="width:50%;"></img>

We now load in data from the 2000-01 and 2018-19 seasons. We'll run this through PCA extracting two components.

In [ ]:
nba = pd.read_csv("nba_team_shots.csv")

In [ ]:
nba.head()

In [ ]:
# Make a PCA object
pca = PCA(n_components = 2)

In [ ]:
# fit it on the desired columns
nba_pca = pca.fit_transform(nba[['zone_' + str(i) + '_perc_att' for i in range(1,16)]])

nba['pca_1'] = nba_pca[:,0]
nba['pca_2'] = nba_pca[:,1]

In addition to the projected values we can examine the projection vectors ($w$ from earlier) called the components.

We'll do this with a sorted data frame, and a heat map.

In [ ]:
pd.DataFrame({'pca_1':pca.components_[0,:],
              'pca_2':pca.components_[1,:]},
            index = ['zone_' + str(i) + '_perc_att' for i in range(1,16)]).sort_values('pca_1')

In [ ]:
pd.DataFrame({'pca_1':pca.components_[0,:],
              'pca_2':pca.components_[1,:]},
            index = ['zone_' + str(i) + '_perc_att' for i in range(1,16)]).sort_values('pca_2')

In [ ]:
# We can visualize with a heatmap
sns.set_style("white")
plt.matshow(pca.components_, cmap = 'plasma')

plt.yticks([0,1], ["First Component", "Second Component"],
          fontsize = 14)

plt.colorbar()

plt.xticks(range(15),
          ['zone_' + str(i) + '_perc_att' for i in range(1,16)], rotation = 90, 
          fontsize = 14)

plt.xlabel("Feature", fontsize = 14)
plt.ylabel("Principal Components", fontsize = 14)

plt.show()

What do these two vectors mean? For any principal component vector, positive (meaning greater than 0) vector entries correspond to more positive principal component values. For example a team with 100% of their shots from Zone 12 will have as negative a first principal component value as possible because that is the most negative row of the first principal component vector table. On the other hand, a team that shot 100% of their shots from Zone 5 will have as positive a first principal component value as possible because that is the most positive row of the first principal component vector table. 

In this way we can read through the principal components and see which features are being picked up in that component. We can also see how this gets reflected in the PCA plot.

In [ ]:
sns.set_style("whitegrid")

plt.figure(figsize=(10,8))

plt.scatter(nba.loc[nba.season == '2000-01','pca_1'].values,
            nba.loc[nba.season == '2000-01','pca_2'].values,
            color = 'orange', label = "2000-01 Season")

plt.scatter(nba.loc[nba.season == '2018-19','pca_1'].values,
            nba.loc[nba.season == '2018-19','pca_2'].values,
            color = 'blue', label = "2018-19 Season")

plt.xlabel("PCA 1", fontsize=16)
plt.ylabel("PCA 2", fontsize=16)

plt.legend(fontsize=14)

plt.show()

This plot tells us that NBA teams from 20 years ago (for the most part) had a much different approach to the game. This reflects the shocking revalation that <a href="https://fivethirtyeight.com/features/how-mapping-shots-in-the-nba-changed-it-forever/">$3$ points is worth more than $2$</a>.

If you're interested in learning more check out my blog post (a shameless plug) on how we can use PCA to track changes in NBA shot distributions over time <a href="http://matthew-osborne.com/mtodata/Posts/PCA-in-NBA.html">http://matthew-osborne.com/mtodata/Posts/PCA-in-NBA.html</a>.

## Questions

<br>
<br>
<br>
<br>

## End Questions

## Practice

Examine the component vectors from the Wisconsin Cancer data. What features about the tumor image seem important?

In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






## How Many Components? The Explained Variance Curve

Another question you may want to answer is how many components should I use?

This is where something called the <i>explained variance curve</i> comes into play. sklearn's PCA function does give us the tools to figure out how much of the variance each principal component explains. This is known as the explained variance ratio of the component and is available via the explained_variance_ratio_ variable. These values indicate the proportion of the original dataset's variance that lies along the axis of each principal component.

In [ ]:
print(pca.explained_variance_ratio_)

Let's return once more to the MNIST data set, the real one.

In [ ]:
X = pd.read_csv("https://raw.githubusercontent.com/cerndb/dist-keras/master/examples/data/mnist.csv")

In [ ]:
pca = PCA(n_components=200)

pca.fit(X[X.columns[1:]])

In [ ]:
plt.figure(figsize=(10,8))

plt.plot(range(1,201),
        np.cumsum(pca.explained_variance_ratio_))

plt.xlabel("PCA Component", fontsize=16)
plt.ylabel("Cumulative Variance Explained", fontsize=16)

plt.show()

In [ ]:
np.shape(X)

As we can see the original data set has $784$ features, however we are able to capture over $80\%$ of the variance with fewer than $50$.

One rule of thumb for choosing the dimension of the hyperplane is to look for the "elbow" in the cumulative explained variance plot. This is where the explained variance ratio stops growing as quickly and is thought of as the "intrinsic dimensionality" of the dataset. This looks to be at around 6 or 7 dimensions for the cancer data set.

An additional feature of `sklearn` PCA is that you can tell it what percentage of the variance you'd like.

In [ ]:
pca = PCA(n_components=.5)

pca.fit(X[X.columns[1:]])

## Practice

Examine the Explained Variance Ratio for the Wisconsin Cancer Data Set.

In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






In [ ]:
# Code here 






That's it for this notebook! I encourage you to check out the Unsupervised Learning Homework where we introduce common extensions of PCA for various types of data as well as show how you can use PCA to address multicolinearity.

See you in Notebook 4 where we introduce Manifold techniques.

## References 

<a href = "https://www.tandfonline.com/doi/abs/10.1080/14786440109462720">On lines and planes of closest fit to systems of points in space</a>

<a href="https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf">Univeristy of Waterloo Matrix Cookbook</a>

<a href="http://www.math.kent.edu/~reichel/courses/monte.carlo/alt4.7d.pdf">Kent State University Notes on Random Vectors and Matrices</a>

<a href="http://www.stat.columbia.edu/~fwood/Teaching/w4315/Fall2009/pca.pdf">Columbia PCA notes</a>

<a href="https://www.stat.cmu.edu/~cshalizi/uADA/12/lectures/ch18.pdf">Central Michigan PCA notes</a>